In [1]:
include("CGS_POMDP.jl");
pomdp = CGSPOMDP();

In [12]:
function visualize_uncertainty(pomdp::CGSPOMDP)
    df = copy(pomdp.state.earth[1].df)
    dom = domain(pomdp.state.earth[1].gt)

    for layer in 1:NUM_LAYERS
        gtlayer = pomdp.state.earth[layer].gt
        data_at_all_wells = gtlayer[Multi([pomdp.collected_locs...]), :]

        γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET) # Each feature can have a different nugget in the future.
        okrig = GeoStatsModels.OrdinaryKriging(γ)
        fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

        for column in pomdp.feature_names
            var_column = Symbol("$(column)_$(layer)_variances")
            df[!, var_column] = [GeoStatsModels.predictprob(fitkrig, column, pt).σ
                            for pt in domain(pomdp.state.earth[1].gt)]
        end
    end
    var_map = georef(df, dom)
    viewer(var_map)
end

visualize_uncertainty (generic function with 2 methods)

### Test: Action function returns all actions

In [3]:
all_actions = POMDPs.actions(pomdp);

### Test: Observation function returns correct distributions

In [4]:
POMDPs.observation(pomdp, all_actions[2], pomdp.state)

DiagNormal(
dim: 15
μ: [0.8453198757256793, 68.17626896778893, 38.38629206652658, 8.374965944397989, 304.80287205453294, 81.40240857950931, 10.555969175330723, 13.914437367397966, 70.50338006675491, 12.123399284044224, 272.87059405893655, 74.49070044875167, 15.30642489627985, 150.90298303462095, 35.64002207040294]
Σ: [0.6617649993262633 0.0 … 0.0 0.0; 0.0 0.6617649993262633 … 0.0 0.0; … ; 0.0 0.0 … 0.6617649993262633 0.0; 0.0 0.0 … 0.0 0.6617649993262633]
)


In [5]:
POMDPs.reward(pomdp, pomdp.state, all_actions[2])

29.0

In [14]:
visualize_uncertainty(pomdp)

In [ ]:
POMDPs.observation(pomdp, all_actions[21], pomdp.state) # Can take up to 7 mins to run!
# Ask Mansur how to speed this up and how to test this

In [ ]:
POMDPs.reward(pomdp, pomdp.state, all_actions[21])

### Test: Points close are predicted with more accuracy and less uncertainty than points far.

In [8]:
gtlayer = pomdp.state.earth[1].gt

# seismic_line = (x1=0.0, y1=0.0, x2=5.0, y2=98.0)
well1 = Point(19, 34)
well2 = Point(30, 12)
well_close = Point(31, 14)
well_far = Point(98, 97)

all_wells = Multi([well1, well2])
data_at_all_wells = gtlayer[all_wells, :]

γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET)

okrig = GeoStatsModels.OrdinaryKriging(γ)
fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

# We notice that prediction at close has much less variance than prediction at far, 
# and as an additional sanity check the prediction at close is close to the well2 point.
# Krigging predictprob only does one attribute at a time.

probs_far = GeoStatsModels.predictprob(fitkrig, :z, well_far)
probs_close = GeoStatsModels.predictprob(fitkrig, :z, well_close)
probs_close, probs_far

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Test: Uncertainty is low around known points.
Additionally, as nugget is increased uncertainty (both globally and at known points) increases.
Pay attention to colorbar when verifying this.

### Buy well data at 3-4 locations and visualize belief

In [10]:
buy_well_data(pomdp, 2)
buy_well_data(pomdp, 5)
buy_well_data(pomdp, 7)

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
visualize_uncertainty(Multi([pomdp.collected_locs...]), 3, "z") # Note this is time consuming at just 3 gathered points.

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
distr = observe(pomdp, Point(18, 24), 3, "z")
rand(distr)

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.